Capítulo 6

6.3 Patrones de organización de datos.

6.3.1 Estructuración jerárquica

Prototipo - Este ejemplo de patrón de organización de datos consiste en que dada una estructura de datos en formato tabla, ésta es convertida a otro tipo de estructura en un formato jerarquizado XML.

In [60]:
%%writefile jerarquico.py
#!/usr/bin/env python
#Usamos el archivo foros.csv

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import RawValueProtocol

import xmlify

class jerarquico(MRJob):
    
    OUTPUT_PROTOCOL = RawValueProtocol
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                reducer_init=self.reducer_init,
                reducer=self.reducer,
                reducer_final=self.reducer_final),
            MRStep(mapper_init=self.mapper_init_dos,
                    mapper=self.mapper_dos,
                  mapper_final=self.mapper_final_dos)
        ]
    def mapper(self,_, line):
        linea=line.split(";") 
        
        mensaje=linea[4] # Recogemos el mensaje de la posición 4 de la línea
        tipoMensaje=linea[5] #Recogemos de la posición 5, si es una pregunta o respuesta
       
        if tipoMensaje=="question": 
            idMensaje=linea[0] #Almacenamos el id único del mensaje
            yield idMensaje,(tipoMensaje,mensaje)
        else:
            idMensaje=linea[7] #Almacenamos el identificador del mensaje idMensaje  
            yield idMensaje,(tipoMensaje,mensaje)
     
    def reducer_init(self):
        self.listaFinal=[]
    def reducer(self, key, values):
        listaValores=[]
        listaPrincipal=[]
        listaAuxiliar=[] 
        
        for v in values: #Metemos los valores que vienen en un matriz
            listaValores.append(v) #Matriz que contiene el tipo de mensaje y el mensaje asociado 
        
        for valor in listaValores:
            if valor[0]=="question":#Si es una pregunta la metemos en la lista principal
                listaPrincipal.append(valor[1])
            else:
                listaAuxiliar.append(valor[1]) # Si son respuestas, las vamos agregando a una lista
        
        #listaPrincipal.append(listaAuxiliar) #agregamos la lista de respuestas a la lista principal
        listaPrincipal.append(listaAuxiliar)
        #self.listaFinal.append(listaPrincipal)
        self.listaFinal.extend(listaPrincipal)
    def reducer_final(self):
        #Conversion a XML indicando en el raiz el id del mensaje
        #yield "Creada linea XML: " ,xmlify.dumps(self.listaFinal,root = "key") 
        #yield "Creada linea XML: " ,xmlify.dumps(self.listaFinal) 
        for valor in self.listaFinal:
            yield "key", valor
    def mapper_init_dos(self):
        self.listaDef=[]
    def mapper_dos(self,key,values):
        self.listaDef.extend(values)
    def mapper_final_dos(self):
        yield "key", xmlify.dumps(self.listaDef,root = "key")

if __name__ == '__main__':
    jerarquico.run()

Overwriting jerarquico.py


In [61]:
!python jerarquico.py archivos_datos/foros_peque.csv > e_jerarquicaV2.xml

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201217.205301.813574
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201217.205301.813574\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201217.205301.813574\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201217.205301.813574...


Capítulo 6

6.3 Patrones de organización de datos.

6.3.1 Estructuración jerárquica

Prototipo - Esta segunda versión vamos a usar la estructura diccionarios de Python

In [6]:
%%writefile jerarquicoV2.py
#!/usr/bin/env python
#Usamos el archivo foros.csv

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
import xmlify

class jerarquicoV2(MRJob):
    OUTPUT_PROTOCOL = RawValueProtocol
    def mapper(self,_, line):
        linea=line.split(";") # Cada línea es un mensaje del foro (pregunta, respuesta o comentario)
        
        mensaje=linea[4] # Recogemos el mensaje de la posición 4 de la línea
        tipoMensaje=linea[5] #Recogemos de la posición 5, si es una pregunta, respuesta o comentario
        
        if tipoMensaje=="question": 
            idMensaje=linea[0] #Almacenamos el id único del mensaje
            yield idMensaje,(tipoMensaje,mensaje)
        else:
            idPadre=linea[7] #Almacenamos el identificador del mensaje idMensaje  
            yield idPadre,(tipoMensaje,mensaje)
     
    def reducer(self, key, values):
        diccionario=dict() #Para el caso que usemos diccionarios
        matrizParaXML=[]
        listaPrincipal=[]
        listaAuxiliar=[]
        
        for v in values: #Metemos los valores que vienen en un matriz
            matrizParaXML.append(v) #Matriz que contiene el tipo de mensaje y el mensaje asociado 
        
        for valor in matrizParaXML:
            if valor[0]=="question":#Si es una pregunta la metemos en la lista principal
                listaPrincipal.append(valor[1])
            else:
                listaAuxiliar.append(valor[1]) # Si son respuestas, las vamos agregando a una lista
        
        listaPrincipal.append(listaAuxiliar) #agregamos la lista de respuestas a la lista principal 
        diccionario[key]=listaPrincipal #Para el caso que usemos diccionarios
        yield key,xmlify.dumps(diccionario) # Conversion a XML para el caso que usemos diccionarios
        
if __name__ == '__main__':
    jerarquicoV2.run()

Writing jerarquicoV2.py


In [7]:
!python jerarquicoV2.py archivos_datos/foros.csv > e_jerarquica.xml

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201211.191824.267729
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201211.191824.267729\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201211.191824.267729\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201211.191824.267729...


In [5]:
%%writefile testJerarquico.py
import unittest
import re,json
from jerarquico import jerarquico

class TestJerarquico(unittest.TestCase):
        
    def setUp(self):
        #Archivo para la unión
        self.archivo = "archivos_datos/foros.csv"
        
        """#self.archivo="clientes.csv"
        self.archivo = "ventas.csv"
        self.diccionario={}
        with open(self.archivo) as f:
            self.tablaEnMemoria = set(line.strip() for line in f)
        for linea in self.tablaEnMemoria:
            encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
            if encontrado==None:
                datos = linea.split(";")
                self.diccionario[datos[0]]=datos"""
        
        #print ("Diccionario relleno: ",self.diccionario) #Veo si el diccionario está relleno
        
    def test_rj(self):
        #self.archivo2=json.dumps(self.diccionario)
        mr_job = jerarquico(['--runner=inline',self.archivo])#Mando al objeto 
                                                                   #MRJOB el diccionario.
                                                                    #ERROR. Si pongo un
                                                                    #archivo, sí funciona.
        with mr_job.make_runner() as runner: #Ejecuto el MRJob
            runner.run()
            for key,value in mr_job.parse_output(runner.cat_output()):
                print (key,value) 
            
if __name__ == '__main__':
    unittest.main()

Overwriting testJerarquico.py


In [6]:
!python testJerarquico.py

No configs specified for inline runner

Error while reading from C:\Users\MRSANC~1\AppData\Local\Temp\jerarquico.mrsanchez.20201215.142852.143053\step\000\reducer\00000\input:

E
ERROR: test_rj (__main__.TestJerarquico)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "testJerarquico.py", line 31, in test_rj
    runner.run()
  File "C:\Users\mrsanchez\Anaconda3\lib\site-packages\mrjob\runner.py", line 503, in run
    self._run()
  File "C:\Users\mrsanchez\Anaconda3\lib\site-packages\mrjob\sim.py", line 161, in _run
    self._run_step(step, step_num)
  File "C:\Users\mrsanchez\Anaconda3\lib\site-packages\mrjob\sim.py", line 170, in _run_step
    self._run_streaming_step(step, step_num)
  File "C:\Users\mrsanchez\Anaconda3\lib\site-packages\mrjob\sim.py", line 187, in _run_streaming_step
    self._run_reducers(step_num, num_reducer_tasks)
  File "C:\Users\mrsanchez\Anaconda3\lib\site-packages\mrjob\sim.py", line 289, in _run